<a href="https://colab.research.google.com/github/stefano2211/DataScienceRedSocial/blob/main/Ridery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
#Importamos las librerias
from textblob import TextBlob
import snscrape.modules.twitter as sntwitter
import plotly.express as px
import neattext as nt
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [175]:
#Cargamos las stopwords de nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [176]:
#Importamos los datos de twitter
query = "@RideryVzla"
tweets = []
limit = 1000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.content, tweet.hashtags, tweet.username, tweet.mentionedUsers])

<ipython-input-176-ccef9170d89e>:13: FutureWarning:

content is deprecated, use rawContent instead

<ipython-input-176-ccef9170d89e>:13: FutureWarning:

username is deprecated, use user.username instead



In [177]:
#Colocamos los datos en un dataframe
df = pd.DataFrame(tweets, columns=['Date', 'Tweet', 'Hashtag', 'Username', 'Mention'])
df.head(100)

,Date,Tweet,Hashtag,Username,Mention
0,2023-02-12 20:03:36+00:00,@Soygordy @RideryVzla 🤣🤣🤣🤣🤣👍,None,gordonesroo,"[https://twitter.com/Soygordy, https://twitter..."
1,2023-02-12 14:11:21+00:00,@gordonesroo @RideryVzla Mano acepto Pago Móvi...,None,Soygordy,"[https://twitter.com/gordonesroo, https://twit..."
2,2023-02-12 13:48:58+00:00,Yo todo el camino en mi moto ridery: Diosito a...,None,erikmargarita,[https://twitter.com/RideryVzla]
3,2023-02-11 22:05:12+00:00,@EchoErGuevon @RideryVzla ¡Gracias! 👍,None,gordonesroo,"[https://twitter.com/EchoErGuevon, https://twi..."
4,2023-02-11 22:02:08+00:00,"@brendilu Hola, Brenda. Leímos que tuviste un ...",None,RideryVzla,[https://twitter.com/brendilu]
...,...,...,...,...,...
95,2023-02-05 01:01:06+00:00,Otro fin de semana que no hay choferes disponi...,None,gordonesroo,[https://twitter.com/RideryVzla]
96,2023-02-04 19:28:29+00:00,Lo mejor que le pudo haber pasado a este país ...,None,callme_lili13,"[https://twitter.com/RideryVzla, https://twitt..."
97,2023-02-04 18:24:10+00:00,"Al cierre de 2022, @RideryVzla presentó su ba...",None,deconveniencia1,"[https://twitter.com/RideryVzla, https://twitt..."
98,2023-02-04 07:09:41+00:00,@RideryVzla help.,None,ori2555,[https://twitter.com/RideryVzla]


In [178]:
#Convertimos la columna Date a formato tiempo
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [179]:
dt = df

In [180]:
dt.head()

,Date,Tweet,Hashtag,Username,Mention
0,2023-02-12 20:03:36+00:00,@Soygordy @RideryVzla 🤣🤣🤣🤣🤣👍,None,gordonesroo,"[https://twitter.com/Soygordy, https://twitter..."
1,2023-02-12 14:11:21+00:00,@gordonesroo @RideryVzla Mano acepto Pago Móvi...,None,Soygordy,"[https://twitter.com/gordonesroo, https://twit..."
2,2023-02-12 13:48:58+00:00,Yo todo el camino en mi moto ridery: Diosito a...,None,erikmargarita,[https://twitter.com/RideryVzla]
3,2023-02-11 22:05:12+00:00,@EchoErGuevon @RideryVzla ¡Gracias! 👍,None,gordonesroo,"[https://twitter.com/EchoErGuevon, https://twi..."
4,2023-02-11 22:02:08+00:00,"@brendilu Hola, Brenda. Leímos que tuviste un ...",None,RideryVzla,[https://twitter.com/brendilu]


In [181]:
#Convertimos el None en Nan y luego remplazamos el Nan por un 1
dt['Hashtag'].replace('None', np.nan, inplace=True)
dt['Hashtag'] = dt['Hashtag'].fillna('1')



In [182]:
dt['Mention'].replace('None', np.nan, inplace=True)
dt['Mention'] = dt['Mention'].fillna('1')

In [183]:
dt.head()

,Date,Tweet,Hashtag,Username,Mention
0,2023-02-12 20:03:36+00:00,@Soygordy @RideryVzla 🤣🤣🤣🤣🤣👍,1,gordonesroo,"[https://twitter.com/Soygordy, https://twitter..."
1,2023-02-12 14:11:21+00:00,@gordonesroo @RideryVzla Mano acepto Pago Móvi...,1,Soygordy,"[https://twitter.com/gordonesroo, https://twit..."
2,2023-02-12 13:48:58+00:00,Yo todo el camino en mi moto ridery: Diosito a...,1,erikmargarita,[https://twitter.com/RideryVzla]
3,2023-02-11 22:05:12+00:00,@EchoErGuevon @RideryVzla ¡Gracias! 👍,1,gordonesroo,"[https://twitter.com/EchoErGuevon, https://twi..."
4,2023-02-11 22:02:08+00:00,"@brendilu Hola, Brenda. Leímos que tuviste un ...",1,RideryVzla,[https://twitter.com/brendilu]


In [184]:
#Extraemos los hashtags para convertirlos en strings
def textl(text):
  leters= " ".join([str(_) for _ in text])
  return leters


In [185]:
dt['TokenMention'] = dt['Mention'].apply(textl)

In [186]:
dt['Token'] = dt['Hashtag'].apply(textl)
dt.head()

,Date,Tweet,Hashtag,Username,Mention,TokenMention,Token
0,2023-02-12 20:03:36+00:00,@Soygordy @RideryVzla 🤣🤣🤣🤣🤣👍,1,gordonesroo,"[https://twitter.com/Soygordy, https://twitter...",https://twitter.com/Soygordy https://twitter.c...,1
1,2023-02-12 14:11:21+00:00,@gordonesroo @RideryVzla Mano acepto Pago Móvi...,1,Soygordy,"[https://twitter.com/gordonesroo, https://twit...",https://twitter.com/gordonesroo https://twitte...,1
2,2023-02-12 13:48:58+00:00,Yo todo el camino en mi moto ridery: Diosito a...,1,erikmargarita,[https://twitter.com/RideryVzla],https://twitter.com/RideryVzla,1
3,2023-02-11 22:05:12+00:00,@EchoErGuevon @RideryVzla ¡Gracias! 👍,1,gordonesroo,"[https://twitter.com/EchoErGuevon, https://twi...",https://twitter.com/EchoErGuevon https://twitt...,1
4,2023-02-11 22:02:08+00:00,"@brendilu Hola, Brenda. Leímos que tuviste un ...",1,RideryVzla,[https://twitter.com/brendilu],https://twitter.com/brendilu,1


In [187]:
tokenHashtag = dt['Token']

In [188]:
tokenMention = dt['TokenMention']

In [189]:
tokens = [token for line in tokenHashtag  for token in line.split()]
tokensMention = [token for line in tokenMention  for token in line.split()]

In [190]:
#Creamos una funcion que cuente los datos ya toquenizados 
def get_tokens(docx,num=40):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [191]:
tokensTotal = get_tokens(tokens)
TokenMentionTotal= get_tokens(tokensMention)

In [192]:
#Insertamos los datos en un dataframe
data = pd.DataFrame(tokensTotal.items(),columns=['Hashtags','scores'])
dataM = pd.DataFrame(TokenMentionTotal.items(),columns=['Mention','scores'])

In [193]:
dataM.head()

,Mention,scores
0,https://twitter.com/RideryVzla,757
1,1,94
2,https://twitter.com/RCamachoVzla,55
3,https://twitter.com/esIsaacF,47
4,https://twitter.com/yummyridesvzla,45


In [194]:
fig = px.bar(dataM, title="Menciones mas usados en la Kings League",x=dataM['Mention'], y=dataM['scores'])
fig.show()

#Conclusiones de los hashtags

Las menciones mas utilizados de Ridery Venezuela


* @RideryVzl 
* @RCamachoVzla
* @esIsaacF

In [195]:
#Ploteamos los datos
fig = px.bar(data, title="Hashtags mas usados en la Kings League",x=data['Hashtags'], y=data['scores'])
fig.show()

#Conclusiones de los hashtags

Los hashtags mas utilizados de la Kings League

1. #Ridery
2. #MotoRidery
3. #Denuncia



In [196]:
#Removemos caracteres que puedan afectar la prediccion del modelo
dt['Tweet'] = dt['Tweet'].apply(nt.remove_hashtags)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_emojis)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_numbers)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_special_characters)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_urls)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_currency_symbols)
dt['Tweet'] = dt['Tweet'].apply(nt.remove_userhandles)

In [197]:
#Creamos una funcion para borrar caracteres especiales
def clean_text(text):
  text = re.sub(r'@[A-Za-z09]+', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/?', '', text)


  return text

dt['Tweet'] = dt['Tweet'].apply(clean_text)

In [198]:
#Creamos dos fuciones una para clacular el sentimiento y el otro para calcular la subjetividad
def get_subjectivity(text):
  return TextBlob(text).translate(from_lang="es", to="en").sentiment.subjectivity

def get_polarity(text):
  return TextBlob(text).translate(from_lang="es", to="en").sentiment.polarity

In [199]:
dt['Subjectivity'] = dt['Tweet'].apply(get_subjectivity)

In [200]:
dt['Polarity'] = dt['Tweet'].apply(get_polarity)

In [201]:
#Creamos una funcion que tome la polaridad y la procese como Positivo, Neutro o negativo
def analysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
     return 'Neutral'
  elif score > 0:
    return 'Positive'

dt['Analysis'] = dt['Polarity'].apply(analysis)

dt.head(800)

,Date,Tweet,Hashtag,Username,Mention,TokenMention,Token,Subjectivity,Polarity,Analysis
0,2023-02-12 20:03:36+00:00,Soygordy RideryVzla,1,gordonesroo,"[https://twitter.com/Soygordy, https://twitter...",https://twitter.com/Soygordy https://twitter.c...,1,0.000000,0.000000,Neutral
1,2023-02-12 14:11:21+00:00,gordonesroo RideryVzla Mano acepto Pago Mvil e...,1,Soygordy,"[https://twitter.com/gordonesroo, https://twit...",https://twitter.com/gordonesroo https://twitte...,1,0.800000,0.600000,Positive
2,2023-02-12 13:48:58+00:00,Yo todo el camino en mi moto ridery Diosito ay...,1,erikmargarita,[https://twitter.com/RideryVzla],https://twitter.com/RideryVzla,1,0.875000,-0.500000,Negative
3,2023-02-11 22:05:12+00:00,EchoErGuevon RideryVzla Gracias,1,gordonesroo,"[https://twitter.com/EchoErGuevon, https://twi...",https://twitter.com/EchoErGuevon https://twitt...,1,0.000000,0.000000,Neutral
4,2023-02-11 22:02:08+00:00,brendilu Hola Brenda Lemos que tuviste un inco...,1,RideryVzla,[https://twitter.com/brendilu],https://twitter.com/brendilu,1,0.433333,0.416667,Positive
...,...,...,...,...,...,...,...,...,...,...
795,2022-12-15 12:29:39+00:00,yordiarteaga RideryVzla Las ubicaciones de con...,1,lanzallanera,"[https://twitter.com/yordiarteaga, https://twi...",https://twitter.com/yordiarteaga https://twitt...,1,0.366667,-0.155556,Negative
796,2022-12-15 03:12:53+00:00,Daniealbarranr RideryVzla Que cagada,1,thesunno,"[https://twitter.com/Daniealbarranr, https://t...",https://twitter.com/Daniealbarranr https://twi...,1,0.800000,-0.200000,Negative
797,2022-12-15 03:11:58+00:00,thesunno RideryVzla Lo tuve que cancelar yo po...,1,Daniealbarranr,"[https://twitter.com/thesunno, https://twitter...",https://twitter.com/thesunno https://twitter.c...,1,0.000000,0.000000,Neutral
798,2022-12-15 02:58:36+00:00,Daniealbarranr RideryVzla Supongo que deberan ...,1,thesunno,"[https://twitter.com/Daniealbarranr, https://t...",https://twitter.com/Daniealbarranr https://twi...,1,0.000000,0.000000,Neutral


In [202]:
count = dt['Analysis'].value_counts()
count

Neutral     492
Positive    381
Negative    127
Name: Analysis, dtype: int64

In [203]:
#Graficamos la polaridad
fig = px.scatter(dt, y=dt['Polarity'])
fig.show()

In [204]:
fig = px.bar(count, title="Sentimientos de los tweets de la kings league jornada 6", color=['red','blue','green'])
fig.show()

#Conclusiones de el analisis de emociones

Podemos concluir que hay mas comentarios negativos que positivos pero lo que mas se resalta es los comentarios neutros con una alta diferencia.


1. Neutrales : 492
2. Positivo: 381
3. Negativos : 127


In [205]:
# Creamos tres sets de datos con los diferentes tipos de comentarios
positive_tweet = dt[dt['Analysis'] == 'Positive']
neutral_tweet = dt[dt['Analysis'] == 'Neutral']
negative_tweet = dt[dt['Analysis'] == 'Negative']

In [206]:
print(stopwords.words('spanish'))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [207]:
#Eliminamos las preposiciones, adverbios y adjetivos
def textdrop(text):

  sw = stopwords.words('spanish')

  text = [word.lower() for word in text.split() if word.lower() not in sw]

  return " ".join(text)

In [ ]:
positive_tweet['Tweet'] = positive_tweet['Tweet'].apply(textdrop)
neutral_tweet['Tweet'] = neutral_tweet['Tweet'].apply(textdrop)
negative_tweet['Tweet'] = negative_tweet['Tweet'].apply(textdrop)

In [209]:
tokenPositive = positive_tweet['Tweet']
tokenNeutral = neutral_tweet['Tweet']
tokenNegative = negative_tweet['Tweet']

In [210]:
#Tokenizamos los tweets
pos_tokens = [token for line in tokenPositive  for token in line.split()]
neut_tokens = [token for line in tokenNeutral  for token in line.split()]
neg_tokens = [token for line in tokenNegative  for token in line.split()]

In [211]:
word_pos = get_tokens(pos_tokens)
word_neu = get_tokens(neut_tokens)
word_neg = get_tokens(neg_tokens)

In [ ]:
word_neu

In [213]:
#Creamos 3 sets de datos con las palabras y el contador 
pos_df = pd.DataFrame(word_pos.items(),columns=['words','scores'])
neut_df = pd.DataFrame(word_neu.items(),columns=['words','scores'])
neg_df = pd.DataFrame(word_neg.items(),columns=['words','scores'])

In [214]:
fig = px.bar(data, title="Palabras mas usadas en los tweets positivos de Ridery",x=pos_df['words'], y=pos_df['scores'])
fig.show()

In [215]:
fig = px.bar(data, title="Palabras mas usadas en los tweets negativos de Ridery",x=neg_df['words'], y=neg_df['scores'])
fig.show()

#Conclusiones de la tokenzacion

Podemos observar que en los tweets positivos las palabras claves que se repiten comunmente es:

1. RideryVzla
2. Ridery
3. Gracias

En los tweets negativo los mas resaltantes son:

1. RideryVzla
2. App 
3. Servicio

In [216]:
fig = px.histogram(dt, x=dt["Date"], title="Tweets durante durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

In [217]:
fig = px.histogram(dt, x=positive_tweet["Date"], title="Tweets positivos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()

In [218]:
fig = px.histogram(dt, x=negative_tweet["Date"], title="Tweets negativos durante el tiempo")
fig.update_layout(bargap=0.2)
fig.show()